# 09.06 Data Scaling

Data does not always come in a way our machine learning algorithms expects it.
We saw that concept a handful of times already:
we extracted extra features (e.g. polynomial features),
we did dimensionality reduction,
we used manifold techniques.
Yet, all these things may still fail against certain data.

![Wine](sl-wine.svg)

<div style="text-align:right;"><sup>sl-wine.svg</sup></div>

And we look at yet another dataset.
We import the usual stuff and the *wine dataset*,
which is an example of data that would fail a simple dimensionality reduction.

The set is made of chemical measurement over three different vineyards in Italy.
One can take it as a classification problem of attempting to identify the
vineyard from the chemical composition.
We will use is for dimensionality reduction.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-talk')
from sklearn.datasets import load_wine
wine = load_wine()
print(wine['DESCR'])

---

One should explore the data first.
We will simply plot as many dimensions as we can at once.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))
ax.scatter(wine.data[:, 0], wine.data[:, 1], s=30*wine.data[:, 2], c=wine.target, cmap='plasma');

The classes seem to be difficult to separate.
Yet, we have just a few dimensions and a handful of samples,
therefore we can perform a full PCA and see whether we can project this data into a different space.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(wine.data)
fig, ax = plt.subplots(figsize=(14, 10))
ax.plot(np.cumsum(pca.explained_variance_ratio_))
ax.set(xlabel='components', ylabel='explained variance');

Oh wow, two dimensional space seem to explain the data variance well enough.
And, since we can visualize a two dimensional space easily, we should do it.

In [ ]:
pca = PCA(n_components=2)
wine_pca = pca.fit_transform(wine.data)
fig, ax = plt.subplots(figsize=(14, 6))
ax.scatter(wine_pca[:, 0], wine_pca[:, 1], s=60, c=wine.target, cmap='plasma');

Despite the fact that we did dimensionality reduction the data does not look separable.
Let's try something different, let's describe this data using `pandas`.

In [ ]:
df = pd.DataFrame(wine.data, columns=[wine.feature_names])
df.describe()

The values of *magnesium* and *proline* have completely different magnitudes from all other features.
These features have much bigger values than all the others,
and since PCA will evaluate variance based on the values alone,
it will take these two features as the main variance explanation.
In other words, instead of finding the main variance in the data PCA is simply finding these two features.

Until now we worked with PCA on images,
in which each dimension follows the same scale:
the possible values of the pixel which are from $0$ to $255$.
Such a well behaved set of dimensions is uncommon in most datasets,
notably if the data are not images.

Let's scale those things down and then apply PCA.
The `StandardScaler` centers the mean of every feature to zero,
and ensures that the variance of each feature is exactly one.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

preprocess = make_pipeline(StandardScaler(), PCA(n_components=2))
wine_pca = preprocess.fit_transform(wine.data)
fig, ax = plt.subplots(figsize=(14, 6))
ax.scatter(wine_pca[:, 0], wine_pca[:, 1], s=60, c=wine.target, cmap='plasma');

Now this is rather easy to separate.
And moreover, we probably do not need a complex classifier for it.

We have two take away messages here.
The first is that if we had believed that the PCA examples we have done
earlier are good representations for all data sets we would be in trouble
when faced with a dataset of non-images.
Not scaling the data before PCA is one of the most common mistakes
one makes when working with data preprocessing.

Another thing that we did not see before is the use of a pipeline
with two preprocessors.
We could now add a third `sklearn` object, perhaps a classifier,
to the pipeline and build a three piece pipeline.
A pipeline is not limited to two `sklearn` objects glued together,
often one may need a longer one.
The dataset after scaling is quite easy to classify,
we will leave the concatenation of a model to the pipeline
and classification to you.

## References

- [SciKit Learn - Importance of Feature Scaling][1]

[1]: http://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html